In [1]:
from os import walk
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
from openpyxl.worksheet.table import Table, TableStyleInfo

In [2]:
class employeeFile():
    def __init__(self):
        self.fileType
        self.fileDate
        self.filePath
    
    # def     
    
    

In [3]:
rawContentDf = pd.DataFrame(columns=['firstName', 'lastName','root','files'])

In [4]:
content = walk("active")
for root, dirs, files in content:
    dirHirarchy = root.split('\\')
    if len(dirHirarchy) == 3 : 
        fullName = dirHirarchy[1].split(', ')
        firstName = fullName[1]
        lastName = fullName[0]
        
        currentContent = pd.DataFrame({'firstName': firstName, 'lastName': lastName, 'root' : root, 'files': [files]}, columns=['firstName', 'lastName','root', 'files'])
        rawContentDf = pd.concat([rawContentDf,currentContent], ignore_index=True)

In [5]:
rawContentDf

,firstName,lastName,root,files
0,firstname1,Lastname1,"active\Lastname1, firstname1\Blue","[medTest2020.pdf, otherMedTest2021.pdf]"
1,firstname1,Lastname1,"active\Lastname1, firstname1\White","[documentation2020.pdf, info2022.pdf, test2023..."
2,firstname2,Lastname2,"active\Lastname2, firstname2\Blue","[medTest2020.pdf, otherMedTest2021.pdf]"
3,firstname2,Lastname2,"active\Lastname2, firstname2\White","[documentation2020.pdf, info2022.pdf, test2023..."
4,firstname3,Lastname3,"active\Lastname3, firstname3\Blue","[medTest2020.pdf, otherMedTest2021.pdf]"
5,firstname3,Lastname3,"active\Lastname3, firstname3\White","[documentation2020.pdf, info2022.pdf, test2023..."


In [6]:

# rawContentDf.groupby(['firstName', 'lastName']).agg({'root': lambda x: x})#.to_csv('output.csv')

In [11]:
excel_path = 'filesRecap.xlsx'

rawContentDf.groupby(['firstName', 'lastName']).agg({'files': lambda x: list(np.concatenate(x.to_numpy())), 'root': lambda x: x.iloc[0]}).to_excel(excel_path)

wb = load_workbook(excel_path)
ws = wb.active

# Step 3: Loop over the rows and add hyperlinks in the third column (C)
for row in range(2, ws.max_row + 1):  # Start from row 2 (because row 1 is the header)
    file_path = ws[f'D{row}'].value  # File path is in column D
    link_description = ws[f'C{row}'].value  # The existing text in column C (Description)
    
    # Create the hyperlink and set it in column C
    ws[f'C{row}'].hyperlink = file_path
    ws[f'C{row}'].value = link_description  # Display text (can also be file_path if you prefer)
    ws[f'C{row}'].font = Font(color="0000FF", underline="single")  # Style the hyperlink text (blue and underlined)

# Step 4: Create an Excel table object
table_range = f"A1:D{ws.max_row}" #columns are hard coded 
table = Table(displayName="FilesTable", ref=table_range)

style = TableStyleInfo(
    name="TableStyleMedium9",  # A pre-built Excel table style
    showFirstColumn=False,
    showLastColumn=False,
    showRowStripes=True,
    showColumnStripes=True
)
table.tableStyleInfo = style

ws.add_table(table)

# Step 5: Save the workbook
wb.save(excel_path)